In [33]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import re
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent
from requests.exceptions import MissingSchema

In [21]:
URL = 'https://hh.ru'
profession = 'Программист'

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(URL)
driver.implicitly_wait(1)
driver.set_page_load_timeout(10)

headers = {
    'Accept': '*/*',
    'User-Agent': UserAgent().chrome
}



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\dm1tr\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "c:\users\dm1tr\desktop\hhru\venv\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "c:\users\dm1tr\desktop\hhru\venv\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [22]:
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

In [23]:
main_block = soup.find_all('div', class_='dashboard-tiles-item__body')

for b in main_block:
    print(b)
    print()

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Вакансии дня</div><div class="dashboard-tiles-item__salary">40 000 – 90 000 руб.</div></div><div class="dashboard-tiles-item__counter"><span>14 вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Компании дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>2038<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Работа из дома</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>37174<!-- --> <!-- -->вакансии</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles

In [24]:
for sub in main_block:
    if sub.find('div', class_='dashboard-tiles-item__title').text == profession:
        profession_selected_block = sub
print(profession_selected_block)

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Программист</div><div class="dashboard-tiles-item__salary">до <!-- -->495 000<!-- --> <!-- -->руб.</div></div><div class="dashboard-tiles-item__counter"><span>18266<!-- --> <!-- -->вакансий</span></div></div>


In [38]:
from selenium.webdriver.support.wait import WebDriverWait

professions = driver.find_elements(by=By.CLASS_NAME, value='dashboard-tiles-item__title')

links_to_profession = list()

if professions:
    professions[0].click()
    professions[0].click()

for elem in professions:
    if elem.text == profession:
        elem.click()
        element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//*[@class='multiple-column-list multiple-column-list_wide']"))
        )
        soup = BeautifulSoup(driver.page_source, 'lxml')
        ul = soup.find(class_='multiple-column-list multiple-column-list_wide')
        for li in ul:
            links_to_profession.append(li.a.get('href'))

In [59]:
def scrap_offer(link, data):
    print(link)
    try:
        req = requests.get(link, headers=headers)
        soup = BeautifulSoup(req.text, parser='lxml')
        title = soup.find('h1', class_='bloko-header-1').text

        reviews_button = soup.find('button', class_='_2kMmwZ7___bloko-link _2iCEfb3___bloko-link_pseudo')
        if reviews_button:
            reviews = reviews_button.span.text.split()[0]
            rating = soup.find('div',
                               class_='_2uXXPFp___bloko-text _3Iu9HPD___bloko-text_extra-large mmwQivW___bloko-text_strong').text
        else:
            reviews = None
            rating = None

        data['reviews_count'].append(reviews)
        data['avg_rating'].append(rating)
        data['title'].append(title)
    except MissingSchema:
        print('Shema error')
        data['salary'].pop(-1)
        data['prof_type'].pop(-1)
    except AttributeError:
        print('None type')
        data['salary'].pop(-1)
        data['prof_type'].pop(-1)

In [60]:
data = {'title': list(), 'prof_type': list(), 'salary': list(), 'reviews_count': list(), 'avg_rating': list()}
page = 0

for link in links_to_profession[:2]:
    req = requests.get(f'{URL}{link}&page={page}', headers=headers)
    print(req.status_code)
    while req.status_code == 200:
        soup = BeautifulSoup(req.text, parser='lxml')

        prof_type = soup.find('h1', class_='bloko-header-section-3').text
        prof_type = re.compile('«.*»').findall(prof_type)[0][1:-1]

        for offer in soup.find_all('div', class_='vacancy-serp-item'):
            raw_salary = offer.find('span', class_='bloko-header-section-3')

            data['prof_type'].append(prof_type)

            if not raw_salary:
                data['salary'].append(None)
            else:
                salary = re.compile('\d+').findall(raw_salary.text.replace(' ', ''))

                if len(salary) > 1:
                    data['salary'].append(f'{salary[0]}-{salary[1]}')
                elif 'от' in raw_salary.text:
                    data['salary'].append(f'от {salary[0]}')
                elif 'до' in raw_salary.text:
                    data['salary'].append(f'до {salary[0]}')
                else:
                    data['salary'].append(None)

            offer_link = offer.find('a', class_='bloko-link').get('href')
            try:
                scrap_offer(offer_link, data)
            except TimeoutException:
                print('Timeout')
                data['salary'].pop(-1)
                data['prof_type'].pop(-1)

        page += 1
        req = requests.get(f'{URL}{link}&page={page}', headers=headers)
    page = 0

200
https://hh.ru/vacancy/54814745?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54712154?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54634168?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54635259?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54798383?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54660649?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%8

KeyboardInterrupt: 

In [61]:
data['salary'].pop(-1)
data['prof_type'].pop(-1)

'Программист 1С'

In [52]:
for k, v in data.items():
    print(len(v))

1358
1358
1358
1358


In [62]:
df = pd.DataFrame(data=data)
df

,title,prof_type,salary,reviews_count,avg_rating
0,Ведущий программист 1С,Программист 1С,200000-250000,None,None
1,Программист 1С,Программист 1С,от 200000,None,None
2,Программист-разработчик (г. Химки),Программист 1С,140000-200000,None,None
3,Программист-консультант 1C,Программист 1С,250000-300000,None,None
4,Программист 1С,Программист 1С,180000-225000,None,None
5,Ведущий программист 1с,Программист 1С,209000-253000,None,None
6,1С ведущий консультант,Программист 1С,от 250000,None,None
7,Программист 1C,Программист 1С,от 3000,9,"4,5"
8,Программист 1С,Программист 1С,230000-280000,None,None
